### Install Dependencies

In [1]:
%pip install utils
%pip install transformers[sentencepiece]
%pip install evaluate
%pip install rouge-score

Note: you may need to restart the kernel to use updated packages.
     -------------------------------------- 904.0/904.0 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.22.3
    Uninstalling protobuf-4.22.3:
      Successfully uninstalled protobuf-4.22.3
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.


     -------------------------------------- 81.4/81.4 kB 912.7 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24972 sha256=953affaeecb9a3b31a756960c4d5bf352bb4f0894d61abc9f4f1113213a1eff5
  Stored in directory: c:\users\brand\appdata\local\pip\cache\wheels\3e\94\5c\7ff8a51c53c1bbc8df4cac58aa4990ffbc6fa203e9f0808fdd
Successfully built rouge-score
Note: you may need to restart the kernel to use updated packages.


### Import

In [142]:
from utils import *
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from sklearn.model_selection import train_test_split
import os
import evaluate
os.environ["WANDB_DISABLED"] = "true"
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

import torch
from torch import cuda
from tqdm import tqdm_notebook as tqdm
device = 'cuda' if cuda.is_available() else 'cpu'
# device = torch.device("cpu")
device





'cpu'

### Load Model and Tokenizer

In [1]:
#model_checkpoint = "google/mt5-small"#This can also be used, though it has different results
model_checkpoint = "sonoisa/t5-base-japanese-v1.1"
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

NameError: ignored

In [210]:
max_input_length = 128
max_target_length = 128

## Dataset

It is just an np.array of many different sentences. y is the target, x is the input. The index of each arrap relate to each other

In [211]:
y = np.array(["最近いろいろあって歌えなかったから、今日は歌いまくった！",
    "どうして俺は彼らのように歌うことが出来ないんだ",
    "人は食事を取らなければ死んでしまう",
    "税金を払ったら実生活の始まりだ。",
    "もう彼女には言えないよ。そんなに単純なことではなくなってきたからね。",
    "お金を稼ぎたいならアメリカが一番だ",
    "最新のバイオ技術によって、芋の臭みkを最小限に抑え、非常に飲みやすい「芋焼酎」です。",
    "あなたは新しいことを勉強したいのだと思った。",
    "野ねずみが農家の穀物を食べ尽くしていった。",
    "とある映画を文庫化した—いや、映画の為に書かれたシナリオを小説として加筆修正し、日本語にローカライズしたものだ。",
    "あの映画はえげつない",
    "両親にこっぴどくしかられてしまいます。",
    "ラザニアが大好きです",
    "あれ？あなたまだここにいたのね！",
    "心配しないで、楽しくいこう！",
    "兵士たちの使命はその橋を破壊hすることだった",
     "今、ようやくツケがまわってきたのだろう",
    "料金は会計係へ払って下さい",
    "僕のおごりですよ。",
    "果樹栽培者がリンゴの接ぎ穂を台木に接ぎ木しました",
    "アップルの新型ノートは予定通りに発表されるだろうか",
    "冷蔵庫のドアを開いたらリンゴが落ちた",
    "林檎は彼女によってナイフで半分に切られた",
    "林檎の形は丸い",
    "緑色である事がその種のりんごの特徴なのです",
    "トップのリーダーは犬の行動学では「アルファと呼ばれ以下「ベータ「ガンマと続きます",
    "こんな時は迎え酒に限ります",
    "時々私の犬は夜中の間間に吠えます",
    "次に私達に犬の世話をする時間があるか今一度一考えてみましょう",
    "下校の途中で犬をつれた女の子と遊んだ",
    "チューたろうは田中さんの犬です",
    "古代のオーバーテクノロジーの武器は必ず制御ができなくなる",
    "シャッターを押すだけで最新のテクノロジーがキレイな一枚に仕上げます",
    "ただの本なら宇宙警察がでしゃばりはしないわ問題はこれがオーバーテクノロジーの塊だってこと",
    "倫理の点から言うと技術関連の大原則は「現在への責任だが、「未来への責任」という新しい考え方が出てきた",
    "この仕事の面白みは、常に技術が進化しているので、刺激を受け続けられることですね",
    "発明や科学技術のために古い語が新しい意味をとるようになる場合が多い",
    "古代のオーバーテクノロジーの武器は必ず制御ができなくなる",
    "シャッターを押すだけで、最新のテクノロジーがキレイな一枚に仕上げます",
    "ただの本なら、宇宙警察がでしゃばりはしないわ、問題はこれがオーバーテクノロジーの塊だってこと",
    "彼は古代史の分野で突っ込んだ研究をしている",
    "民主主義は古代ギリシャに始まった",
    "父は古代史に関心を持っている",
    "彼は船で何世紀も前に自分の土地を離れた古代の神様についての話を聞いていた",
    "彼は古代神話に基づく小説を書いた",
    "彼は古代史に精通している",
    "何か重い兇器でやられたらしく、頭蓋骨は粉砕された",
    "万が一に備えて、武器になるものを探した",
    "秘密兵器の知識がある人は、我が身を危険な立場に置いている",
    "彼は私に武器を向けた",
    "彼は傘を武器として使った",
    "原子爆弾は恐ろしい武器だ",
    "世界的な全面核戦争が起これば、地球規模でこの「核の冬」が数カ月間も続くと言われています",
    "問題は、いかに核戦争を避けるかである",
    "放射能が原子力発電所から漏れた",
    "文明は今や核戦争に脅かされている",
    "彼らは声を大にして核実験反対を叫んだ",
    "彼らは核兵器のことを気にかけているようだ",
    "生物学は好きになれません",
    "聞いたところでは、彼は生物を勉強するために渡米したそうだ",
    "彼女は生物学の学位を持っている",
    "彼は生物研究所に１０年近く従事している",
    "彼は生物学の分野で研究している",
    "彼は生物学に非常に関心をもっている",
    "省エネのためにコンビニの２４時間営業を廃止しろ",
    "暖房と同様に、冷房時の消費電力量を測定し、旧型と省エネ型（２００１年製）のエアコンを比較しました",
    "ヘレンの言葉で私は急に力づいた",
    "「ねこまっしぐら」というのは、飼い猫がエサに向かってまいしんする様子を表している",
    "成歩堂先生の事務所が「エネルギー分野における世界有数の法律事務所」である",
    "たまにはビール以外のお酒をちびちび舐めるのも良いでしょう",
    "マックで軽く昼食をとって、スタバでコーヒーを飲んで、しゃぶしゃぶをしながら飲み会という流れだ",
    "こちらの飲み物は単品となります",
    "休みの前などは少し羽目を外して飲むのだが、杜仲茶割りで飲むと二日酔いが全くない",
    "父は普通ビールを飲むが、今夜はワインを飲んだ",
    "父は蒸留酒は飲まない",
    "医者に相談して鎮静剤を処方してもらうのはどうでしょうか",
    "診察室を出る間際、先生が「バイバイと手を振ってくださいました",
    "近医で処方を希望したら露骨に嫌な顔をされた",
    "うん前の先生に紹介してもらって、転院したの",
    "「あ、は、はい・・・ごめん、玲姉」「コラ幾ら親戚とはいえ、私は先輩医師よ院内ではちゃんとケジメをつけなさい」",
    "医者になれる日はまだほど遠い",
    "何かしてみましょう",
    "この世界の教育にはがっかりしてしまう",
    "「信用して」と彼は言った",
    "失礼だが、上記の記事にある３つの誤りを指摘しておきたい",
    "あの方が言われる事を、何でもしてあげて下さい",
    "彼は彼の人生の話を私にしてくれた",
    "脅しによるリーダーシップは、いまぼくたちが目にするような結果を生み出しはしない",
    "照明はツマミ一つで光量を変えられるようになっていて好きな明るさを演出出来るようになっている",
    "文法的に正しい文章を作るよう心がけるべきだ",
    "彼らは同じ製品をはるかにやすい原価で製造できる",
    "農民は作物を作る",
    "農家の人々は温室で作物を育てざるを得ないのです",
    "そうしたなかで急速な進歩を遂げてきたのが、である",
    "ママあのお姉ちゃんとお兄ちゃん、お手手繋いで、すっごく仲良しさんだね",
    "普通、アマチュアでは250ヤード飛べばすごい飛距離だと言われます",
    "皆さんは、モラルを大切にしていますか",
    "いわゆる裏方仕事もたくさんあります",
    "これは世界の常識であり、資源管理の大原則だ",
    "そういう相手の前では、私の見せ掛けだけの怜悧な技術は、見抜かれた時に負けていたと思う",
    "利口な生徒はそうした質問に簡単に答えられる",
    "利口な学生達は早くテストを終えた",
    "利口な学生であればそのような事はしないだろう",
    "非常に賢くおとなしいので、この犬は私にとって良い友達です",
    "彼女は利口というよりむしろ賢明である",
    "赤道における経度1度当たりの長さ",
    "昨日ゆっくり休んだ分、今日は頭が冴えている",
    "その会社では高い専門性が要求される",
    "容疑者は自白するまできびしい尋問を受けた",
    "僕は彼に少なからず恩義がある",
    "彼女は博士号を取得した",
    "弊社は機械パーツの輸入を行っています",
    "彼女は米国史を専攻するだろう",
    "彼は大学で物理学を専攻することに決めた",
    "前の会社では何でも屋に終始したので今度は専門職に就きたい",
    "私は中世史を専攻している",
    "今後、生物学を専攻する学生の数は増加するであろう",
    "あなたは大学で何を専攻されましたか",
    "アルミニウム、マグネシウム、シリコンなどの粉末が爆発する",
    "怒鳴りつけたかったが思いとどまった",
    "心臓が、どきどきして破裂しそう",
    "蒸気ボイラーは爆発する可能性がある",
    "そのニュースは彼の怒りを爆発させた",
    "このゲームの目的は画面にあるすべての爆弾を爆発させることです",
    "説明してもいいけど、君の脳みそ爆発するよ",
    "勉強のしすぎで頭が爆発しそう",
    "怒りが爆発する前に、ガス抜きしなきゃ"])

X = np.array(["さいきんいろいろあってうたえなかtたから、きょうはうたいまくった！",
    "どうしておれはかれらにのようにうたうことができないんだ。",
    "ひとはしょくじをとらなければしんでしまう",
    "ぜいきんをはらたらじっせいかつのはじまりだ。",
    "もうかのじょにはいえないよ。そんなにたんじゅんなことではなくなてきたからね。",
    "おかねをかせぎたいならアメリカがいちばんだ",
    "さいしんのバイオぎじゅつによtて、いものくさみをさいしょうげんにおさえ、ひじょうにのみやすい「いもしょうちゅう」です。",
    "あなたはあたらしいことをべんきょうしたいのだとおもった。",
    "のねずみがのうかのこくもつをたべつくしていった。",
    "とあるえいがをぶんこかした—いや、えいがのためにかかれたシナリオをしょうせつとしてかひつしゅうせいし、にほんごにロオカライズしたものだ。",
    "あのえいがはえげつない",
    "りょうしんにこっぴどくしかられてしまいます。",
    "ラザニアがだいすきです",
    "あれ？あなたまだここにいたのね！",
    "しんぱいしないで、たのしくいこう！",
    "へいしたちのしめいはそのはしをはかいすることだった",
    "いま、ようやくツケがまわってきたのだろう",
    "りょうきんはかいけいがかりへはらってください",
    "ぼくのおごりですよ",
    "かじゅさいばいしゃがリンゴのはぎほをだいぎにつぎきしました",
    "アっプルのしんがたノオトはよていどおりにはっぴょうされるだろうか",
    "れいぞうこのドアをひらいたらリンゴがおちた",
    "りんごはかのじょによtてナイフではんぶんにきられた",
    "りんごのかたちはまるい",
    "りょくしょくであることがそのたねのりんごのとくちょうなのです",
    "トっプのリイダアはいぬのこうどうがくではアルフアとよばれいかベエタガンマとつずきます",
    "こんなときはむかえざけにかぎります",
    "ときどきわたしのいぬはのにほえます",
    "にわたしたちにいぬのせわをするじかんがあるかいまどかんがえてみましょう",
    "げこうのとちゅうでいぬをつれたおんなのことあそんだ",
    "チュウたろはたなかさんのいぬです",
    "こだいのオオバアテクノロジイのぶきはかならずせいぎょができなくなる",
    "シャっタアをおすだけでさいしんのテクノロジイがキレイなにしあげます",
    "ただのほんならうちゅうけいさつがでしゃばりはしないわもんだいはこれがオオバアテクノロジイのかたまりだってこと",
    "りんりのてんからいうとぎじゅつかんれんのだいげんそくはげんざいへのせきにんだがみらいへのせきにんというあたらしいかんがえかたがでてきた",
    "このしごとのおもしろみはつねにぎじゅつがしんかしているのでしげきをうけつずけられることですね",
    "はつめいやかがくぎじゅつのためにふるいかたりがあたらしいいみをとるようになるばあいがおおい",
    "こだいのオオバアテクノロジイのぶきはかならずせいぎょができなくなる",
    "シャっタアをおすだけでさいしんのテクノロジイがキレイなにしあげます",
    "ただのほんならうちゅうけいさつがでしゃばりはしないわもんだいはこれがオオバアテクノロジイのかたまりだってこと",
    "かれはこだいしのぶん'やでつっこんだけんきゅうをしている",
    "みんしゅしゅぎはこだいギリシャにはじまった",
    "ちちはこだいしにかんしんをもっている",
    "かれはふねでなにせいきもまえにじぶんのとちをはなれたこだいのかみさまについてのはなしをきいていた",
    "かれはこだいしんわにもとずくしょうせつをかいた",
    "かれはこだいしにせいつうしている",
    "なにかおもいきょうきでやられたらしくずがいこつはふんさいされた",
    "まんがいちにそなえてぶきになるものをさがした",
    "ひみつへいきのちしきがあるひとはわがみをきけんなたちばにおいている",
    "かれはわたしにぶきをむけた",
    "かれはかさをぶきとしてつかった",
    "げんしばくだんはおそろしいぶきだ",
    "せかいてきなぜんめんかくせんそうがおこればちきゅうきぼでこのかくのふゆがすうかげつかんもつずくといわれています",
    "もんだいはいかにかくせんそうをさけるかである",
    "ほうしゃのうがげんしりょくはつでんしょからもれた",
    "ぶんめいはいまやかくせんそうにおびやかされている",
    "かれらはこえをだいにしてかくじっけんはんたいをさけんだ",
    "かれらはかくへいきのことをきにかけているようだ",
    "せいぶつがくはすきになれません",
    "きいたところではかれはせいぶつをべんきょうするためにとべいしたそうだ",
    "かのじょはせいぶつがくのがくいをもっている",
    "かれはせいぶつけんきゅうじょに１０ねんちかくじゅうじしている",
    "かれはせいぶつがくのぶんやでけんきゅうしている",
    "かれはせいぶつがくにひじょうにかんしんをもっている",
    "しょうえねのためにコンビニの２４えいぎょうをはいししろ",
    "だんぼうとどうようにれいぼうじのしょうひでんりょくりょうをそくていしきゅうがたしょうえね（２００１ねんせい）のエアコンをひかくしました",
    "ヘレンのことばでわたしはきゅうにちからずいた",
    "ねこまっしぐらというのはかいねこがエサにむかってまいしんするようすをあらわしている",
    "なるふどうせんせいのじむしょがエネルギイぶんやにおけるせかいゆうすうのほうりつじむしょである",
    "たまにはビイルいがいのおさけをちびちびなめるのもよいでしょう",
    "マックでかるくちゅうしょくをとってスタバでコオヒイをのんでしゃぶしゃぶをしながらのみかいというながれだ",
    "こちらののみものはたんぴんとなります",
    "やすみのまえなどはすこしはめをはずしてのむのだがもりなかちゃわりでのむとふつかよいがまったくない",
    "ちちはふつうビイルをのむがこん'やはワインをのんだ",
    "ちちはじょうりゅうしゅはのまない",
    "いしゃにそうだんしてちんせいざいをしょほうしてもらうのはどうでしょうか",
    "しんさつしつをでるまぎわせんせいがバイバイとてをふってくださいました",
    "きんいでしょほうをきぼうしたらろこつにいやなかおをされた",
    "うんまえのせんせいにしょうかいしてもらっててんいんしたの",
    "あははい・・・ごめんれいあねコラいくらしんせきとはいえわたしはせんぱいいしよいんないではちゃんとケジメをつけなさい",
    "いしゃになれるひはまだほどとおい",
    "なにかしてみましょう",
    "このせかいのきょういくにはがっかりしてしまう",
    "しんようしてとかれはいった",
    "しつれいだがじょうきのきじにある３みっつのあやまりをしてきしておきたい",
    "あのほうがいわれることをなにでもしてあげてください",
    "かれはかれのじんせいのはなしをわたしにしてくれた",
    "おどしによるリイダアシっプはいまぼくたちがめにするようなけっかをうみだしはしない",
    "しょうめいはツマミひとつでひかりりょうをかえられるようになっていてすきなあかるさをえんしゅつできるようになっている",
    "ぶんぽうてきにただしいぶんしょうをつくるようこころがけるべきだ",
    "かれらはおなじせいひんをはるかにやすいげんかでせいぞうできる",
    "のうみんはさくもつをつくる",
    "のうかのひとびとはおんしつでさくもつをそだてざるをえないのです",
    "そうしたなかできゅうそくなしんぽをとげてきたのがイっである",
    "ママあのおねえちゃんとおにいちゃのてしゅつないですっごくなかよしさんだね",
    "ふつうアマチュアでは250ヤアドとべばすごいひきょりだといわれます",
    "みなさんはモラルをたいせつにしていますか",
    "いわゆるうらかたしごともたくさなります",
    "これはせかいのじょうしきでありしげんかんりのだいげんそくだ",
    "そういうあいてのまえではわたしのみせかけだけのれいりなぎじゅつはみぬかれたときにまけていたとおもう",
    "りこうなせいとはそうしたしつもんにかんたんにこたえられる",
    "りこうながくせいたちははやくテストをおえた",
    "りこうながくせいであればそのようなことはしないだろう",
    "ひじょうにかしこくおとなしいのでこのいぬはわたしにとってよいともだちです",
    "かのじょはりこうというよりむしろけんめいである",
    "せきどうにおけるけいど1あたりのながさ",
    "きのうゆっくりやすんだぶんきょうはあたまがさえている",
    "そのかいしゃではたかいせんもんせいがようきゅうされる",
    "ようぎしゃはじはくするまできびしいじんもんをうけた",
    "ぼくはかれにすくなからずおんぎがある",
    "かのじょははかせごうをしゅとくした",
    "へいしゃはきかいパアツのゆにゅうをおこなっています",
    "かのじょはべいこくしをせんこうするだろう",
    "かれはだいがくでぶつりがくをせんこうすることにきめた",
    "まえのかいしゃではなにでもやにしゅうししたのでこんどはせんもんしょくにつきたい",
    "わたしはちゅうせいしをせんこうしている",
    "こんごせいぶつがくをせんこうするがくせいのかずはぞうかするであろう",
    "あなたはだいがくでなにをせんこうされましたか",
    "アルミニウムマグネシウムシリコンなどのふんまつがばくはつする",
    "どなりつけたかったがおもいとどまった",
    "しんぞうがどきどきしてはれつしそう",
    "じょうきボイラアはばくはつするかのうせいがある",
    "そのニュウスはかれのいかりをばくはつさせた",
    "このゲエムのもくてきはがめんにあるすべてのばくだんをばくはつさせることです",
    "せつめいしてもいいけどきみののうみそばくはつするよ",
    "べんきょうのしすぎであたまがばくはつしそう",
    "いかりがばくはつするまえにガスぬきしなきゃ"])

#X = np.array(X)
##y = np.array(y)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((101,), (26,), (101,), (26,))

In [212]:
class MultiLabelDataset(torch.utils.data.Dataset):

    def __init__(self, text, labels, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.text = text
        self.labels = labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer(
            text,
            max_length=self.max_len,
            truncation=True,
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
#         token_type_ids = inputs["token_type_ids"]
        
        text = self.labels[index][0]
        labels = self.tokenizer(
            text,
            max_length=self.max_len,
            return_token_type_ids=True
        )
        
#         print(inputs)

        return {
            'input_ids': torch.tensor(ids, dtype=torch.long),
            'attention_mask': torch.tensor(mask, dtype=torch.long),
#             'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'labels': torch.tensor(labels['input_ids'], dtype=torch.long)
#                 'token_type_ids': torch.tensor(labels['token_type_ids'], dtype=torch.long),
        }

#### Tokenize the Dataset & Hyperparams

In [213]:
tokenized_datasets = {}
tokenized_datasets['train'] = MultiLabelDataset(X_train, y_train, tokenizer, max_input_length)
tokenized_datasets['validation'] = MultiLabelDataset(X_test, y_test, tokenizer, max_input_length)

In [216]:
batch_size = 32
num_train_epochs = 3
# Show the training loss with every epoch
logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

In [217]:
args = Seq2SeqTrainingArguments(
    output_dir=f"/data/{model_name}-japanese",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    push_to_hub=False,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [218]:

rouge_score = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=False)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=False)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [219]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [220]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

### Train!

In [221]:
trainer.train()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,14.571500,11.359753,11.152800,3.238900,10.641900,10.579300
2,14.523800,11.079609,9.564100,1.619400,9.400800,9.345400
3,14.469900,10.980495,15.072700,7.180600,14.402900,14.379000


TrainOutput(global_step=12, training_loss=14.090277353922525, metrics={'train_runtime': 237.4203, 'train_samples_per_second': 1.276, 'train_steps_per_second': 0.051, 'total_flos': 14928245250048.0, 'train_loss': 14.090277353922525, 'epoch': 3.0})

Save Model if wanted

In [222]:
trainer.save_model("./kana_model")

In [223]:
kana_model = AutoModelForSeq2SeqLM.from_pretrained("./kana_model")

# Test Model. Put a Japanese sentence in the text and pass it through the tokenizer and run the cell below

In [227]:
text = "ひとはしょくじをとらなければしんでしまう"
inputs = tokenizer(text, return_tensors="pt")   
print(inputs)
#outputs = kana_model.generate(inputinput_ids)
outputs = trainer.model.generate(input_ids=inputs["input_ids"],attention_mask=inputs["attention_mask"],do_sample=False,)
print(outputs)

print(tokenizer.decode(outputs[0], skip_special_tokens=False))

{'input_ids': tensor([[  261,  5120,   266, 27422,   782,   269,   273,   394,  3450,  3107,
           272,  4701,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
tensor([[    0, 32099,   287,   261, 32075,   260,   261,  5120,   266, 27422,
           782,   269,   273,   394,  3450,  3107,   272,  4701,   260, 15011]])
<pad><extra_id_0>し <extra_id_24>。 ひとはしょくじをとらなければしんでしまう。こころ
